# Einfaches Lineares Programm zur Stahlproduktion

Ein einfaches lineares Programm zur Stahlproduktion wird hier aufgestellt und algorithmisch gelöst. 

$$
	\begin{array}{lll}
	\max & 600s_1+650s_2+500s_3 &\\
	\textrm{unter} & 2s_1+s_2+1,5s_3&\leq 20000 \\
	 & 0,5s_1+2s_2+1,5s_3 &\leq 10000\\
	 & 0,05s_1+0,08s_2 +0,01s_3 &\leq 1000  \\
	 & 10s_1+12s_2 +8s_3 &\leq 5000  \\
	 & s_1,s_2,s_3&\geq 0
	\end{array}
$$

Wir verwenden dafür die Bibliothek <code>mip</code>, die sowohl in der Lage ist lineare Programme wie auch mixed integer programs zu lösen. Das Paket mip muss vorher installiert werden. Wenn Sie google colab verwenden, so können Sie mip wie unten in der Zelle installieren. Wenn Sie jupyter / python auf Ihrem eigenen Rechner laufen lassen, dann installieren Sie mip durch den Befehl <code>pip install mip</code>, den Sie in einem Terminal ausführen. 

Dokumentation zum mip-Paket findet sich hier: https://python-mip.readthedocs.io/en/latest/

Das mip-Paket ist im Wesentlichen die Klebe zwischen dem eigentlich solver und der Anwendung. Das bedeutet insbesondere, dass das mip-Paket mit verschiedenen solvern kombiniert werden kann. Für uns ist das aber ersteinmal unerheblich -- mip kommt zusammen mit einem solver, der absolut ausreichend ist.

Wenn Sie in Google Colab arbeiten, dann verwenden Sie die nächste Zelle so wie sie ist. Wenn Sie auf dem eigenen Rechner arbeiten und <code>mip</code> bereits installiert haben, dann löschen Sie die nächste Zelle oder kommentieren Sie per Raute den Installationsbefehl.

In [1]:
# für google colab
!pip install mip  # Löschen / Auskommentieren, wenn Sie mip bereits installiert haben.

 Nach dem Installieren müssen wir Python sagen, dass wir die Pakete verwenden wollen: dies geschieht mit <code>import paket_name</code>.

In [2]:
import mip

<code>mip.Model</code> ist die zentrale Klasse, die sowohl das mip speichert als auch für die Lösung zuständig ist.

In [3]:
model=mip.Model()

(Was macht der Punkt zwischen <code>mip</code> und <code>Model()</code>? Damit sagen wir Python, dass wir auf die Klasse namens <code>Model</code> zugreifen wollen, die sich im Paket <code>mip</code> befindet.)

Wir legen drei Variablen an. 

$s_1\geq 0, s_2\geq 0, s_3\geq 0$

Die Variablen werden dem Model mit der Methode <code>add_var</code> hinzugefügt. Die Methode akzeptiert verschiedene Parameter. Wir nutzen hier 
den Parameter <code>lb</code>, der eine untere Schranke für die Variablen festlegt. (Tatsächlich wäre dies hier unnötig, da per default die untere Schranke bei 0 liegt.)
Eine obere Schranke von 20000 ließe sich mit <code>ub=20000</code> anlegen. Die Funktion <code>model.add_var</code> liefert ein Objekt zurück -- dieses weisen wir einer (Python-)Variable mit <code>s1=...</code> zu, damit wir weiter mit den Variablen des LPs arbeiten können.

In [4]:
s1=model.add_var(lb=0)
s2=model.add_var(lb=0)
s3=model.add_var(lb=0)

Als nächstes kommt die Zielfunktion dran: 

$\max 600s_1+650s_2+500s_3$

Die Zielfunktion wird durch den Parameter <code>model.objective</code> festgelegt -- dabei muss dem Modell noch mitgeteilt werden, ob maximiert oder minimiert wird. Dies geschieht mit den Methoden <code>mip.maximize</code> und <code>mip.minimize</code>. Die Zielfunktion selbst können wir ganz so schreiben wie oben im linearen Programm.

In [5]:
model.objective=mip.maximize(600*s1+650*s2+500*s3)

Bedingungen lassen sich einfach zum Model hinzufügen.

$2s_1+s_2+s3\leq 20000$

Die Syntax ist hier einfach, die Bedingung wird dem Modell hinzu addiert. Wir nutzen dabei den Operator <code>model+=...</code>, der eine Abkürzung für <code>model = model + ...</code> ist.

In [6]:
model+= 2*s1+s2+s3 <= 20000

Drei weitere Bedingungen...

$0.5s_1+2s_2+1.5s_3\leq 10000$

$0.05s_1+0.08s_2+0.01s_3\leq 1000$

$10s_1+12s_2+8s_3\leq 5000$

In [7]:
model+= 0.5*s1+2*s2+1.5*s3 <= 10000
model+= 0.05*s1+0.08*s2+0.01*s3 <= 1000
model+= 10*s1+12*s2+8*s3 <= 5000

Wir starten den eigentlichen Algorithmus -- die Rückmeldung zeigt, dass das Optimum gefunden wurde. 

In [8]:
model.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming problem using Primal Simplex



<OptimizationStatus.OPTIMAL: 0>

Ausgabe der Lösung sowie des Zielfunktionswertes. An die Werte der Variablen kommen wir mit <code>Variablenname.x</code> heran. Dh, nach der Lösung gibt <code>s1.x</code> den optimalen Wert der Variable s1 wider. Die Methode <code>objective_value</code> liefert den optimalen Zielfunktionswert zurück.

In [9]:
print("s1={}, s2={}, s3={}".format(s1.x,s2.x,s3.x))
print("Zielfunktionswert: {}".format(model.objective_value))

s1=0.0, s2=0.0, s3=624.9999999999999
Zielfunktionswert: 312499.99999999994


Wie man sieht, gibt es gewisse Rundungsungenauigkeiten. (Man sieht auch, dass das lineare Programm ein nicht so interessantes Problem beschreibt: Tatsächlich muss nur die maximale Menge der dritten Stahlsorte produziert werden.)

Das mip-Paket ist in der Lage mit verschiedenen solvern zu arbeiten. Welcher solver gerade verwendet wurde, können wir uns ausgeben lassen:

In [10]:
model.solver_name

'CBC'

<code>optimize</code> liefert je nach Verlauf der Optimierung verschiedene Werte zurück. Welche möglich sind, können wir uns angucken:

In [11]:
help(mip.constants.OptimizationStatus)

Help on class OptimizationStatus in module mip.constants:

class OptimizationStatus(enum.Enum)
 |  OptimizationStatus(value, names=None, *, module=None, qualname=None, type=None, start=1, boundary=None)
 |  
 |  Status of the optimization
 |  
 |  Method resolution order:
 |      OptimizationStatus
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  CUTOFF = <OptimizationStatus.CUTOFF: 7>
 |  
 |  ERROR = <OptimizationStatus.ERROR: -1>
 |  
 |  FEASIBLE = <OptimizationStatus.FEASIBLE: 3>
 |  
 |  INFEASIBLE = <OptimizationStatus.INFEASIBLE: 1>
 |  
 |  INT_INFEASIBLE = <OptimizationStatus.INT_INFEASIBLE: 4>
 |  
 |  LOADED = <OptimizationStatus.LOADED: 6>
 |  
 |  NO_SOLUTION_FOUND = <OptimizationStatus.NO_SOLUTION_FOUND: 5>
 |  
 |  OPTIMAL = <OptimizationStatus.OPTIMAL: 0>
 |  
 |  OTHER = <OptimizationStatus.OTHER: 10000>
 |  
 |  UNBOUNDED = <OptimizationStatus.UNBOUNDED: 2>
 |  
 |  ---------------------------------------------------